# Annihilation operator for a single boson

The cutoff L needs to be a power of 2 because we want to use qubit operators.

In [1]:
import numpy as np
from scipy.sparse import diags
L = 8  # cutoff for Fock space
a = diags(np.sqrt(np.linspace(1,L-1,L-1)),offsets=1)

In [2]:
print(a.toarray())

[[0.         1.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         1.41421356 0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         1.73205081 0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         2.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         2.23606798
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  2.44948974 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         2.64575131]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]


## Representation in QISKIT

In [3]:
from qiskit.opflow import MatrixOp

qubitOp = MatrixOp(primitive=a)
print(qubitOp.num_qubits)

3


In [4]:
type(qubitOp)

qiskit.opflow.primitive_ops.matrix_op.MatrixOp

In [5]:
print(qubitOp)

Operator([[0.        +0.j, 1.        +0.j, 0.        +0.j, 0.        +0.j,
           0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
          [0.        +0.j, 0.        +0.j, 1.41421356+0.j, 0.        +0.j,
           0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
          [0.        +0.j, 0.        +0.j, 0.        +0.j, 1.73205081+0.j,
           0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
          [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
           2.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
          [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
           0.        +0.j, 2.23606798+0.j, 0.        +0.j, 0.        +0.j],
          [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
           0.        +0.j, 0.        +0.j, 2.44948974+0.j, 0.        +0.j],
          [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
           0.      

Express the annihilation operator of a single boson in terms of N-qubit operators represented by Pauli matrices (2x2)

In [6]:
a_pauli = qubitOp.to_pauli_op()
print(a_pauli)

SummedOp([
  0.9517337620166573 * IIX,
  0.9517337620166573j * IIY,
  0.4829629131445341 * IXX,
  -0.4829629131445341j * IXY,
  0.4829629131445341j * IYX,
  0.4829629131445341 * IYY,
  -0.14271676764170982 * IZX,
  -0.14271676764170982j * IZY,
  0.25 * XXX,
  -0.25j * XXY,
  -0.25j * XYX,
  -0.25 * XYY,
  0.25j * YXX,
  0.25 * YXY,
  0.25 * YYX,
  -0.25j * YYY,
  -0.2687210601244379 * ZIX,
  -0.2687210601244379j * ZIY,
  -0.12940952255126034 * ZXX,
  0.12940952255126034j * ZXY,
  -0.12940952255126034j * ZYX,
  -0.12940952255126034 * ZYY,
  -0.040295934250509535 * ZZX,
  -0.040295934250509535j * ZZY
])


In [7]:
type(a_pauli)

qiskit.opflow.list_ops.summed_op.SummedOp

In [8]:
a_pauli.to_matrix()

array([[0.        +0.j, 1.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 1.41421356+0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 1.73205081+0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        2.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 2.23606798+0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 2.44948974+0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 2.

# Identity operator for a single boson

In [9]:
from scipy.sparse import identity
iden = identity(L)

In [10]:
qubitOp = MatrixOp(primitive=iden)
print(qubitOp.num_qubits)

3


In [11]:
i_pauli = qubitOp.to_pauli_op()
print(i_pauli)

III


In [12]:
i_pauli.to_matrix()

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

# Create the Hamiltonian for 6 interacting bosons

Try to use the information in [this QISKIT tutorial](https://qiskit.org/documentation/tutorials/operators/01_operator_flow.html)

In [13]:
Nmat = 6  # BMN2 for SU(2) has 6 bosonic matrices

In [14]:
from qiskit.opflow import ListOp, TensoredOp

boson = ListOp([i_pauli]*6)
print(boson)

ListOp([
  III,
  III,
  III,
  III,
  III,
  III
])


In [15]:
print(boson[0])

III


In [16]:
type(boson[0])

qiskit.opflow.primitive_ops.pauli_op.PauliOp

We can use a list of `ListOp` and then perform the tensor product using `TensoredOp`

In [17]:
# generically speaking, we construct the list of bosons and then take the outer product
a_list = []  # this will contain a1...a6 as a list of ListOp
for i in np.arange(0,Nmat):  # loop over all operators
    operator_list = [i_pauli] * Nmat  # only the identity repeated Nmat times
    operator_list[i] = a_pauli  # the i^th element is now the annihilation operator for a single boson
    a_list.append(ListOp(operator_list))


In [18]:
print(a_list[0])

ListOp([
  SummedOp([
    0.9517337620166573 * IIX,
    0.9517337620166573j * IIY,
    0.4829629131445341 * IXX,
    -0.4829629131445341j * IXY,
    0.4829629131445341j * IYX,
    0.4829629131445341 * IYY,
    -0.14271676764170982 * IZX,
    -0.14271676764170982j * IZY,
    0.25 * XXX,
    -0.25j * XXY,
    -0.25j * XYX,
    -0.25 * XYY,
    0.25j * YXX,
    0.25 * YXY,
    0.25 * YYX,
    -0.25j * YYY,
    -0.2687210601244379 * ZIX,
    -0.2687210601244379j * ZIY,
    -0.12940952255126034 * ZXX,
    0.12940952255126034j * ZXY,
    -0.12940952255126034j * ZYX,
    -0.12940952255126034 * ZYY,
    -0.040295934250509535 * ZZX,
    -0.040295934250509535j * ZZY
  ]),
  III,
  III,
  III,
  III,
  III
])


In [19]:
type(a_list[0])

qiskit.opflow.list_ops.list_op.ListOp

In [20]:
# here we create a list of operators which are the tensor products...
a_tensor = [TensoredOp(x) for x in a_list]


In [21]:
print(a_tensor[0])

TensoredOp([
  SummedOp([
    0.9517337620166573 * IIX,
    0.9517337620166573j * IIY,
    0.4829629131445341 * IXX,
    -0.4829629131445341j * IXY,
    0.4829629131445341j * IYX,
    0.4829629131445341 * IYY,
    -0.14271676764170982 * IZX,
    -0.14271676764170982j * IZY,
    0.25 * XXX,
    -0.25j * XXY,
    -0.25j * XYX,
    -0.25 * XYY,
    0.25j * YXX,
    0.25 * YXY,
    0.25 * YYX,
    -0.25j * YYY,
    -0.2687210601244379 * ZIX,
    -0.2687210601244379j * ZIY,
    -0.12940952255126034 * ZXX,
    0.12940952255126034j * ZXY,
    -0.12940952255126034j * ZYX,
    -0.12940952255126034 * ZYY,
    -0.040295934250509535 * ZZX,
    -0.040295934250509535j * ZZY
  ]),
  III,
  III,
  III,
  III,
  III
])


In [22]:
print(a_tensor[0].to_pauli_op())

0.9517337620166573 * IIXIIIIIIIIIIIIIII
- 0.2687210601244379 * ZIXIIIIIIIIIIIIIII
- 0.14271676764170982 * IZXIIIIIIIIIIIIIII
- 0.040295934250509535 * ZZXIIIIIIIIIIIIIII
+ 0.9517337620166573j * IIYIIIIIIIIIIIIIII
+ -0.2687210601244379j * ZIYIIIIIIIIIIIIIII
+ -0.14271676764170982j * IZYIIIIIIIIIIIIIII
+ -0.040295934250509535j * ZZYIIIIIIIIIIIIIII
+ 0.4829629131445341 * IXXIIIIIIIIIIIIIII
- 0.12940952255126034 * ZXXIIIIIIIIIIIIIII
+ 0.4829629131445341j * IYXIIIIIIIIIIIIIII
+ -0.12940952255126034j * ZYXIIIIIIIIIIIIIII
+ -0.4829629131445341j * IXYIIIIIIIIIIIIIII
+ 0.12940952255126034j * ZXYIIIIIIIIIIIIIII
+ 0.4829629131445341 * IYYIIIIIIIIIIIIIII
- 0.12940952255126034 * ZYYIIIIIIIIIIIIIII
+ 0.25 * XXXIIIIIIIIIIIIIII
+ 0.25j * YXXIIIIIIIIIIIIIII
+ -0.25j * XYXIIIIIIIIIIIIIII
+ 0.25 * YYXIIIIIIIIIIIIIII
+ -0.25j * XXYIIIIIIIIIIIIIII
+ 0.25 * YXYIIIIIIIIIIIIIII
- 0.25 * XYYIIIIIIIIIIIIIII
+ -0.25j * YYYIIIIIIIIIIIIIII


The representation is already in terms of Pauli operators.

In [23]:
a_tensor[0].num_qubits

18

In [24]:
sparse = a_tensor[0].to_spmatrix()
type(sparse)

scipy.sparse.csr.csr_matrix

In [25]:
sparse.shape

(8, 8)

In [26]:
# from qiskit.utils import algorithm_globals
# algorithm_globals.massive=True  # needed to big matrices...
# full = a_tensor[0].to_matrix()
# len(full)

ValueError: 'to_matrix' will return an exponentially large matrix, in this case '262144x262144' elements. Set algorithm_globals.massive=True or the method argument massive=True if you want to proceed.

In [ ]:
# full.shape

(4096, 4096)

We also create the identity in this new tensor space of 6 bosons

In [27]:
i_tensor = TensoredOp(ListOp([i_pauli] * Nmat))

In [28]:
type(i_tensor)

qiskit.opflow.list_ops.tensored_op.TensoredOp

In [29]:
print(i_tensor)

TensoredOp([
  III,
  III,
  III,
  III,
  III,
  III
])


In [30]:
i_tensor.to_pauli_op()

PauliOp(Pauli('IIIIIIIIIIIIIIIIII'), coeff=1.0)

In [31]:
from qiskit.opflow import I

I^Nmat

PauliOp(Pauli('IIIIII'), coeff=1.0)

In [ ]:
i_tensor.num_qubits

6

In [ ]:
# full = i_tensor.to_matrix()
# len(full)

4096

In [32]:
print(i_tensor.to_spmatrix())

  (0, 0)	(1+0j)
  (1, 1)	(1+0j)
  (2, 2)	(1+0j)
  (3, 3)	(1+0j)
  (4, 4)	(1+0j)
  (5, 5)	(1+0j)
  (6, 6)	(1+0j)
  (7, 7)	(1+0j)


## Create the position operators

In [33]:
# for each boson they are constructed using a and adag
x_tensor = [1/np.sqrt(2)*(~x + x) for x in a_tensor]

In [34]:
print(x_tensor[0])

0.7071067811865475 * SummedOp([
  TensoredOp([
    SummedOp([
      0.9517337620166573 * IIX,
      -0.9517337620166573j * IIY,
      0.4829629131445341 * IXX,
      0.4829629131445341j * IXY,
      -0.4829629131445341j * IYX,
      0.4829629131445341 * IYY,
      -0.14271676764170982 * IZX,
      0.14271676764170982j * IZY,
      0.25 * XXX,
      0.25j * XXY,
      0.25j * XYX,
      -0.25 * XYY,
      -0.25j * YXX,
      0.25 * YXY,
      0.25 * YYX,
      0.25j * YYY,
      -0.2687210601244379 * ZIX,
      0.2687210601244379j * ZIY,
      -0.12940952255126034 * ZXX,
      -0.12940952255126034j * ZXY,
      0.12940952255126034j * ZYX,
      -0.12940952255126034 * ZYY,
      -0.040295934250509535 * ZZX,
      0.040295934250509535j * ZZY
    ]),
    III,
    III,
    III,
    III,
    III
  ]),
  TensoredOp([
    SummedOp([
      0.9517337620166573 * IIX,
      0.9517337620166573j * IIY,
      0.4829629131445341 * IXX,
      -0.4829629131445341j * IXY,
      0.4829629131445341j * IYX,

In [35]:
print(x_tensor[0].to_pauli_op())

SummedOp([
  1.3459547940123242 * IIXIIIIIIIIIIIIIII,
  -0.38002896772325595 * ZIXIIIIIIIIIIIIIII,
  -0.20183198837695568 * IZXIIIIIIIIIIIIIII,
  -0.056987056725565095 * ZZXIIIIIIIIIIIIIII,
  0.6830127018922192 * IXXIIIIIIIIIIIIIII,
  -0.18301270189221924 * ZXXIIIIIIIIIIIIIII,
  0.6830127018922192 * IYYIIIIIIIIIIIIIII,
  -0.18301270189221924 * ZYYIIIIIIIIIIIIIII,
  0.35355339059327373 * XXXIIIIIIIIIIIIIII,
  0.35355339059327373 * YYXIIIIIIIIIIIIIII,
  0.35355339059327373 * YXYIIIIIIIIIIIIIII,
  -0.35355339059327373 * XYYIIIIIIIIIIIIIII
])


In [36]:
print(x_tensor[0].to_spmatrix())

  (0, 1)	(0.7071067811865475+0j)
  (1, 0)	(0.7071067811865475+0j)
  (1, 2)	(0.9999999999999998+0j)
  (2, 1)	(0.9999999999999998+0j)
  (2, 3)	(1.2247448713915892+0j)
  (3, 2)	(1.2247448713915892+0j)
  (3, 4)	(1.414213562373095+0j)
  (4, 3)	(1.414213562373095+0j)
  (4, 5)	(1.5811388300841893+0j)
  (5, 4)	(1.5811388300841893+0j)
  (5, 6)	(1.732050807568877+0j)
  (6, 5)	(1.732050807568877+0j)
  (6, 7)	(1.8708286933869713+0j)
  (7, 6)	(1.8708286933869713+0j)


## Create the full Hamiltonian

In [37]:
from qiskit.opflow import SummedOp

In [38]:
H_zero = 0.5*Nmat*i_tensor

In [39]:
print(H_zero)

3.0 * TensoredOp([
  III,
  III,
  III,
  III,
  III,
  III
])


In [40]:
H_zero_pauli = 0.5*Nmat*(I^Nmat)

In [41]:
print(H_zero.to_pauli_op())

3.0 * IIIIIIIIIIIIIIIIII


In [42]:
print(H_zero_pauli)

3.0 * IIIIII


In [43]:
H_zero.to_spmatrix()

<8x8 sparse matrix of type '<class 'numpy.complex128'>'
	with 8 stored elements in Compressed Sparse Row format>

In [44]:
H_zero.to_pauli_op()

PauliOp(Pauli('IIIIIIIIIIIIIIIIII'), coeff=3.0)

In [45]:
### Harmonic oscillator
# this should be summed over all the bosons (Nmat)
H_list = [H_zero]
for op in a_tensor:
    H_list.append((~op @ op))
H_osc = SummedOp(H_list)


In [46]:
print(H_osc)

SummedOp([
  3.0 * TensoredOp([
    III,
    III,
    III,
    III,
    III,
    III
  ]),
  ComposedOp([
    TensoredOp([
      SummedOp([
        0.9517337620166573 * IIX,
        -0.9517337620166573j * IIY,
        0.4829629131445341 * IXX,
        0.4829629131445341j * IXY,
        -0.4829629131445341j * IYX,
        0.4829629131445341 * IYY,
        -0.14271676764170982 * IZX,
        0.14271676764170982j * IZY,
        0.25 * XXX,
        0.25j * XXY,
        0.25j * XYX,
        -0.25 * XYY,
        -0.25j * YXX,
        0.25 * YXY,
        0.25 * YYX,
        0.25j * YYY,
        -0.2687210601244379 * ZIX,
        0.2687210601244379j * ZIY,
        -0.12940952255126034 * ZXX,
        -0.12940952255126034j * ZXY,
        0.12940952255126034j * ZYX,
        -0.12940952255126034 * ZYY,
        -0.040295934250509535 * ZZX,
        0.040295934250509535j * ZZY
      ]),
      III,
      III,
      III,
      III,
      III
    ]),
    TensoredOp([
      SummedOp([
        0.951733762

In [47]:
type(H_osc)

qiskit.opflow.list_ops.summed_op.SummedOp

In [48]:
H_osc_pauli = H_osc.to_pauli_op()

In [49]:
type(H_osc_pauli)

qiskit.opflow.list_ops.summed_op.SummedOp

In [50]:
H_osc_pauli.num_qubits

18

In [51]:
H_osc.to_spmatrix()

<8x8 sparse matrix of type '<class 'numpy.complex128'>'
	with 8 stored elements in Compressed Sparse Row format>

In [52]:
quartic1 = SummedOp([x_tensor[2]@x_tensor[2]@x_tensor[3]@x_tensor[3], \
                    x_tensor[2]@x_tensor[2]@x_tensor[4]@x_tensor[4], \
                    x_tensor[1]@x_tensor[1]@x_tensor[3]@x_tensor[3], \
                    x_tensor[1]@x_tensor[1]@x_tensor[5]@x_tensor[5], \
                    x_tensor[0]@x_tensor[0]@x_tensor[4]@x_tensor[4], \
                    x_tensor[0]@x_tensor[0]@x_tensor[5]@x_tensor[5]])
                    

In [ ]:
print(quartic1.to_pauli_op())

SummedOp([
  13.499999999999998 * IIIIIIIIIIII,
  -1.4999999999999993 * ZIIIIIIIIIII,
  -1.4999999999999998 * ZZIIIIIIIIII,
  -1.4999999999999993 * IIZIIIIIIIII,
  -1.4999999999999998 * IIZZIIIIIIII,
  -1.4999999999999993 * IIIIZIIIIIII,
  -1.4999999999999998 * IIIIZZIIIIII,
  -1.4999999999999993 * IIIIIIZIIIII,
  0.24999999999999978 * IIZIIIZIIIII,
  0.2499999999999999 * IIZZIIZIIIII,
  0.24999999999999978 * IIIIZIZIIIII,
  0.2499999999999999 * IIIIZZZIIIII,
  -1.4999999999999998 * IIIIIIZZIIII,
  0.2499999999999999 * IIZIIIZZIIII,
  0.25 * IIZZIIZZIIII,
  0.2499999999999999 * IIIIZIZZIIII,
  0.25 * IIIIZZZZIIII,
  -1.4999999999999993 * IIIIIIIIZIII,
  0.24999999999999978 * ZIIIIIIIZIII,
  0.2499999999999999 * ZZIIIIIIZIII,
  0.24999999999999978 * IIIIZIIIZIII,
  0.2499999999999999 * IIIIZZIIZIII,
  -1.4999999999999998 * IIIIIIIIZZII,
  0.2499999999999999 * ZIIIIIIIZZII,
  0.25 * ZZIIIIIIZZII,
  0.2499999999999999 * IIIIZIIIZZII,
  0.25 * IIIIZZIIZZII,
  -1.4999999999999993 * IIIIIIII

In [53]:
quartic2 = SummedOp([x_tensor[0]@x_tensor[2]@x_tensor[3]@x_tensor[5],\
                    x_tensor[0]@x_tensor[1]@x_tensor[3]@x_tensor[4],\
                    x_tensor[1]@x_tensor[2]@x_tensor[4]@x_tensor[5]], coeff = -2.0)

In [ ]:
print(quartic2.to_pauli_op())

SummedOp([
  -1.7410254037844382 * IXIXIIIXIXII,
  0.46650635094610937 * ZXIXIIIXIXII,
  0.46650635094610937 * IXZXIIIXIXII,
  -0.1249999999999999 * ZXZXIIIXIXII,
  0.4665063509461094 * IXIXIIZXIXII,
  -0.1249999999999999 * ZXIXIIZXIXII,
  -0.1249999999999999 * IXZXIIZXIXII,
  0.0334936490538903 * ZXZXIIZXIXII,
  0.4665063509461094 * IXIXIIIXZXII,
  -0.12499999999999989 * ZXIXIIIXZXII,
  -0.12499999999999989 * IXZXIIIXZXII,
  0.0334936490538903 * ZXZXIIIXZXII,
  -0.1249999999999999 * IXIXIIZXZXII,
  0.0334936490538903 * ZXIXIIZXZXII,
  0.0334936490538903 * IXZXIIZXZXII,
  -0.008974596215561343 * ZXZXIIZXZXII,
  -0.9012210650134379 * XXIXIIIXIXII,
  -0.9012210650134379 * YYIXIIIXIXII,
  0.24148145657226694 * XXZXIIIXIXII,
  0.24148145657226694 * YYZXIIIXIXII,
  0.24148145657226694 * XXIXIIZXIXII,
  0.24148145657226694 * YYIXIIZXIXII,
  -0.06470476127563014 * XXZXIIZXIXII,
  -0.06470476127563014 * YYZXIIZXIXII,
  0.24148145657226697 * XXIXIIIXZXII,
  0.24148145657226697 * YYIXIIIXZXII,
 

In [ ]:
type(quartic2)

qiskit.opflow.list_ops.summed_op.SummedOp

In [54]:
### Quartic Interaction
V =  quartic1 + quartic2

In [55]:
V.num_qubits

18

In [56]:
V_pauli = V.to_pauli_op()

It takes only 6 seconds to create the Pauli version of this interaction terms when $\Lambda=8$

## Define the 'tHooft coupling

In [57]:
g2N = 0.2
N = 2

In [58]:
### Full Hamiltonian
H = H_osc + g2N / N * V

In [59]:
type(H)

qiskit.opflow.list_ops.summed_op.SummedOp

In [ ]:
H.to_pauli_op()

In [ ]:
type(H.to_pauli_op())

qiskit.opflow.list_ops.summed_op.SummedOp

In [ ]:
H.num_qubits

12

In [ ]:
testH = H.to_matrix_op()
testH

If we do not do this conversion for each term, we do not get the correct result!

In [60]:
Hpauli = H.to_pauli_op()

# the line below is also fine...
# Hpauli1 = H_osc.to_pauli_op() + g2N / N * V.to_pauli_op()
# Hpauli

It takes about 6 seconds and only 2GB of memory to store the Pauli form of the Hamiltonian when $\Lambda = 8$.

In [27]:
Hpauli

NameError: name 'Hpauli' is not defined

In [62]:
type(Hpauli)

qiskit.opflow.list_ops.summed_op.SummedOp

In [61]:
# write expression to disk
with open('pauliH_L8_bos2.txt','w') as f:
    print(Hpauli,file=f)

# Diagonalize the Hamiltonian to find the groundstate

In [23]:
from qiskit.algorithms import NumPyMinimumEigensolver

npme = NumPyMinimumEigensolver()

In [24]:
# result = npme.compute_minimum_eigenvalue(operator=H_osc)
# result = npme.compute_minimum_eigenvalue(operator=H)
result = npme.compute_minimum_eigenvalue(operator=Hpauli)
# result = npme.compute_minimum_eigenvalue(operator=testH)

In [25]:
print(result)

{   'aux_operator_eigenvalues': None,
    'eigenstate': VectorStateFn(Statevector([-9.47620420e-01+3.18627009e-01j,
              1.56136406e-15+6.94220991e-16j,
             -6.22180317e-17+3.62471582e-16j,
             -9.94443076e-18+1.36881516e-18j,
             -2.90751075e-16+4.98240902e-16j,
             -1.65345909e-17-3.36397420e-17j,
              1.90365419e-17+9.85324542e-18j,
              4.06879268e-17-7.72321349e-17j,
              7.61188596e-16-1.36148630e-16j,
              7.61838939e-19-1.28229029e-17j,
              7.76694124e-18-4.11000098e-18j,
              2.52222711e-17-1.17089097e-16j,
              1.24015117e-17-1.86904173e-17j,
              3.06830741e-17+1.40022570e-16j,
              5.04204953e-17-1.39861574e-16j,
             -8.25443523e-17+7.36772904e-17j,
              9.51380258e-16-5.75015622e-16j,
              2.77831646e-17-1.99751691e-18j,
              3.38281552e-18-4.18611871e-19j,
             -1.16077939e-16+1.08713337e-16j,
          

In [26]:
ref_value = result.eigenvalue.real
print(f'Ground state energy value: {ref_value:.5f}')

Ground state energy value: 3.14808


I can not get anything different from 3.000 even if I have the interactions turned on... The reason I found out is that the `MatrixOp` is interpreted differently than a `SummedOp` by the eigensolver. When using the `SummedOp` that I get from the `PauliOp` procedure, it turns out that the statevector is just living in a 2-dimensional space (1 qubit) instead of the full tensor product space of 6 qubits:
``` 
dims=(2,)) instead of dims=(2, 2, 2, 2, 2, 2))
```

One solution is to create the Hamiltonian calling `.to_pauli_op()` on each term.

## Construct the Hamiltonian matrix as usual
This only works for cutoffs smaller than 5 due to memory requirements in `MatrixOp`.

In [27]:
from scipy.sparse import kron 

def build_operators(L: int, N: int) -> list:
    """Generate all the annihilation operators needed to build the hamiltonian
    Args:
        L (int): the cutoff of the single site Fock space
        N (int): the number of colors of gauge group SU(N)
    Returns:
        list: a list of annihilation operators, length=N_bos
    """
    # The annihilation operator for the single boson
    a_b = diags(np.sqrt(np.linspace(1, L - 1, L - 1)), offsets=1)
    # The identity operator of the Fock space of a single boson
    i_b = identity(L)
    # Bosonic Hilbert space
    N_bos = int(2 * (N ** 2 - 1))  # number of boson sites -> fixed for mini-BMN 2
    product_list = [i_b] * N_bos  # only the identity for bosons repeated N_bos times
    a_b_list = []  # this will contain a1...a6
    for i in np.arange(0, N_bos):  # loop over all bosonic operators
        operator_list = product_list.copy()  # all elements are the identity operator
        operator_list[
            i
        ] = a_b  # the i^th element is now the annihilation operator for a single boson
        a_b_list.append(
            operator_list[0]
        )  # start taking tensor products from first element
        for a in operator_list[1:]:
            a_b_list[i] = kron(
                a_b_list[i], a
            )  # do the outer product between each operator_list element
    return a_b_list

In [28]:
a_b_list = build_operators(L,N)
# Build the Hamiltonian
# Start piece by piece
x_list = []
# only use the bosonic operators
for op in a_b_list:
    x_list.append(1 / np.sqrt(2) * (op.conjugate().transpose() + op))
# Free Hamiltonian
H_k = 0
for a in a_b_list:
    H_k = H_k + a.conjugate().transpose() * a
# vacuum energy
H_k = H_k + 0.5 * Nmat * identity(L ** Nmat)

In [23]:
am = MatrixOp(a_b_list[0])

In [24]:
# this takes a really long time even for L=4
print(am.to_pauli_op())

SummedOp([
  0.5 * XIIIII,
  0.5j * YIIIII
])


The operation of using `PauliOp` directly to construct a single boson annihilation operator in the full space gives
```
0.5 * XIIIII
+ 0.5j * YIIIII
```


In [25]:
xm = MatrixOp(x_list[0])

In [26]:
print(xm.to_pauli_op())

0.7071067811865477 * XIIIII


The corresponding `PauliOp` is
```
0.7071067811865475 * XIIIII
```

### The result of computing the energy using the free Hamiltonian as a `MatrixOp`

In [33]:
Hm = MatrixOp(H_k)
result = npme.compute_minimum_eigenvalue(operator=Hm)

In [34]:
ref_value = result.eigenvalue.real
print(f'Ground state energy value: {ref_value:.5f}')

Ground state energy value: 3.00000


### Add interactions

In [29]:
v_b1 = ( x_list[2] * x_list[2] * x_list[3] * x_list[3]
    + x_list[2] * x_list[2] * x_list[4] * x_list[4]
    + x_list[1] * x_list[1] * x_list[3] * x_list[3]
    + x_list[1] * x_list[1] * x_list[5] * x_list[5]
    + x_list[0] * x_list[0] * x_list[4] * x_list[4]
    + x_list[0] * x_list[0] * x_list[5] * x_list[5]
    )
v_b2 = -2.0 * ( x_list[0] * x_list[2] * x_list[3] * x_list[5]
    + x_list[0] * x_list[1] * x_list[3] * x_list[4]
    + x_list[1] * x_list[2] * x_list[4] * x_list[5]
    )
V_b = v_b1 + v_b2


In [71]:
vm = MatrixOp(v_b1)

In [73]:
vm.to_pauli_op()

PauliOp(Pauli('IIIIII'), coeff=1.4999999999999998)

In [74]:
vm = MatrixOp(v_b2)

In [75]:
vm.to_pauli_op()

SummedOp([PauliOp(Pauli('IXXIXX'), coeff=-0.49999999999999994), PauliOp(Pauli('XIXXIX'), coeff=-0.49999999999999994), PauliOp(Pauli('XXIXXI'), coeff=-0.49999999999999994)], coeff=1.0, abelian=False)

In [30]:
# full hamiltonian
H_full = H_k + g2N / N * V_b

In [31]:
Hm = MatrixOp(H_full)
Hm.to_pauli_op()

SummedOp([PauliOp(Pauli('IIIIII'), coeff=6.150000000000001), PauliOp(Pauli('IIIIIZ'), coeff=-0.5000000000000004), PauliOp(Pauli('IIIIZI'), coeff=-0.5000000000000009), PauliOp(Pauli('IIIZII'), coeff=-0.5), PauliOp(Pauli('IIZIII'), coeff=-0.5), PauliOp(Pauli('IXXIXX'), coeff=-0.05), PauliOp(Pauli('IZIIII'), coeff=-0.5), PauliOp(Pauli('XIXXIX'), coeff=-0.05), PauliOp(Pauli('XXIXXI'), coeff=-0.05), PauliOp(Pauli('ZIIIII'), coeff=-0.5)], coeff=1.0, abelian=False)

Using `PauliOp` with L=2 I get
```
SummedOp([PauliOp(Pauli('IIIIII'), coeff=6.150000000000002), PauliOp(Pauli('ZIIIII'), coeff=-0.5), PauliOp(Pauli('IZIIII'), coeff=-0.5), PauliOp(Pauli('IIZIII'), coeff=-0.5), PauliOp(Pauli('IIIZII'), coeff=-0.5), PauliOp(Pauli('IIIIZI'), coeff=-0.5), PauliOp(Pauli('IIIIIZ'), coeff=-0.5), PauliOp(Pauli('XXIXXI'), coeff=-0.04999999999999999), PauliOp(Pauli('XIXXIX'), coeff=-0.04999999999999999), PauliOp(Pauli('IXXIXX'), coeff=-0.04999999999999999)], coeff=1.0, abelian=False)
```


In [32]:
result = npme.compute_minimum_eigenvalue(operator=Hm)
ref_value = result.eigenvalue.real
print(f'Ground state energy value: {ref_value:.5f}')

Ground state energy value: 3.14808


In [33]:
result = npme.compute_minimum_eigenvalue(operator=Hm.to_pauli_op())
ref_value = result.eigenvalue.real
print(f'Ground state energy value: {ref_value:.5f}')

Ground state energy value: 3.14808


In [34]:
print(result)

{   'aux_operator_eigenvalues': None,
    'eigenstate': VectorStateFn(Statevector([-6.53152306e-01+7.56901379e-01j,
              4.39091153e-16-2.51286056e-16j,
             -5.47069374e-16+9.13964942e-16j,
              2.31451472e-17-4.45336007e-17j,
             -1.70463142e-17-5.47824537e-16j,
              3.84882473e-17-1.01862779e-17j,
              3.62020762e-17+5.32859159e-18j,
             -3.40307766e-17-2.95363325e-17j,
             -9.91365094e-16-2.26594666e-16j,
             -2.04203158e-16+6.62687331e-18j,
             -3.71877473e-17-7.39322672e-18j,
             -2.85230492e-17+5.47882114e-18j,
             -8.43919020e-17-1.02496506e-16j,
             -1.46231434e-17+1.23676908e-19j,
              3.05682332e-18-2.05368745e-17j,
             -6.28212538e-18+1.24113009e-18j,
             -7.70784566e-16+3.40654814e-16j,
              2.99513357e-18+2.98985451e-17j,
             -1.62555824e-16-2.72969079e-17j,
             -3.64690913e-17+3.00817293e-18j,
          